In [1]:
import os
import sys
import shutil
import time
import pytz
import traceback

import Ska.engarchive.fetch as fetch
import Chandra.Time

import datetime as dt
import matplotlib.dates as mdate
from matplotlib import gridspec

import matplotlib.pyplot as plt
import mpld3

import numpy as np
import pandas as pd
from astropy.table import Table

from Scripts.msidlists import *
from Scripts.event_times import *
from Scripts.plot_stylers import *

from tqdm import tqdm as progressbar

from hrcsentinel import hrccore as hrc


# allow_subset=True should let us draw more data points
# fetch.data_source.set('maude allow_subset=True')
fetch.data_source.set('cxc')
# fetch.data_source.set('maude')

hrc.styleplots()
labelsizes = 15
plt.rcParams['axes.titlesize'] = labelsizes + 2
plt.rcParams['axes.labelsize'] = labelsizes
plt.rcParams['xtick.labelsize'] = labelsizes
plt.rcParams['ytick.labelsize'] = labelsizes

%config InlineBackend.figure_format = 'retina'

ModuleNotFoundError: No module named 'Scripts'

In [ ]:
temperature_msids

In [2]:
fetch.data_source.set('cxc') # just use ska for this

In [3]:
def convert_to_doy(datetime_start):
    '''
    Return a string like '2020:237' that will be passed to start= in fetch.get_telem()
    '''

    year = datetime_start.year
    day_of_year = datetime_start.timetuple().tm_yday
    doystring = '{}:{}'.format(year, day_of_year)

    return doystring

In [4]:
data_start = '2020:303'
data_start

'2020:303'

lets go one 4 days post anomaly:

In [5]:
data_stop = '2020:341'

Fetch the data

In [6]:
dat = fetch.get_telem(temperature_msids, start=data_start, stop=data_stop, sampling='5min')

NameError: name 'temperature_msids' is not defined

In [56]:
fetch.data_source.set('cxc') 

msids = ['2CHTRPZT']

data_start = '2020:237'
data_stop = '2020:238'

dat = fetch.get_telem(msids, start=data_start, stop=data_stop)


columns = [dat["2CHTRPZT"].times, 
           mdate.num2date(hrc.convert_chandra_time(dat["2CHTRPZT"].times)),        
          dat["2CHTRPZT"].vals, 
          ]

column_names = ["Chandra Time",
                "Human-readable Time (UTC)",
               "CEA EED Temperature (C)",  ]

a= pd.DataFrame(data=columns).T
a.columns=column_names
a

Fetching full-resolution data for MSIDS=['2CHTRPZT']
  from 2020:237:12:00:00.000 to 2020:238:12:00:00.000


,Chandra Time,Human-readable Time (UTC),CEA EED Temperature (C)
0,7.14658e+08,2020-08-24 12:01:11.769435+00:00,35.6503
1,7.14658e+08,2020-08-24 12:01:28.169437+00:00,35.6503
2,7.14658e+08,2020-08-24 12:01:44.569438+00:00,35.6503
3,7.14658e+08,2020-08-24 12:02:00.969440+00:00,35.6503
4,7.14658e+08,2020-08-24 12:02:17.369441+00:00,35.6503
5,7.14658e+08,2020-08-24 12:02:33.769443+00:00,35.6503
6,7.14658e+08,2020-08-24 12:02:50.169445+00:00,35.6503
7,7.14658e+08,2020-08-24 12:03:06.569446+00:00,35.6503
8,7.14658e+08,2020-08-24 12:03:22.969448+00:00,35.6503
9,7.14658e+08,2020-08-24 12:03:39.369439+00:00,35.6503


In [9]:
dat

MSIDset([('2FE00ATM',
          <MSID start=2020:237:12:00:00.000 stop=2020:238:12:00:00.000 len=0 dtype=float32 stat=5min>),
         ('2LVPLATM',
          <MSID start=2020:237:12:00:00.000 stop=2020:238:12:00:00.000 len=0 dtype=float32 stat=5min>),
         ('2IMHVATM',
          <MSID start=2020:237:12:00:00.000 stop=2020:238:12:00:00.000 len=0 dtype=float32 stat=5min>),
         ('2IMINATM',
          <MSID start=2020:237:12:00:00.000 stop=2020:238:12:00:00.000 len=0 dtype=float32 stat=5min>),
         ('2SPHVATM',
          <MSID start=2020:237:12:00:00.000 stop=2020:238:12:00:00.000 len=0 dtype=float32 stat=5min>),
         ('2SPINATM',
          <MSID start=2020:237:12:00:00.000 stop=2020:238:12:00:00.000 len=0 dtype=float32 stat=5min>),
         ('2PMT1T',
          <MSID start=2020:237:12:00:00.000 stop=2020:238:12:00:00.000 len=264 dtype=float32 unit=DEGC stat=5min>),
         ('2PMT2T',
          <MSID start=2020:237:12:00:00.000 stop=2020:238:12:00:00.000 len=264 dtype=flo

In [10]:
def make_excel_datestring(date):
    date.replace(tzinfo=None)
    print(type(date))
    epoch = dt.datetime(1899, 12,30) # Excel epoch is 1900-01-00
    epoch.replace(tzinfo=None)
    delta = date - epoch
    return float(delta.days) + (float(delta.seconds) / 86400)

In [11]:
columns = [dat["2PMT1T"].times, 
           mdate.num2date(hrc.convert_chandra_time(dat["2PMT1T"].times)),        
          dat["2PMT1T"].vals, 
          dat["2PMT2T"].vals,
          dat["2DTSTATT"].vals,
          dat["2DCENTRT"].vals,
          dat["2FHTRMZT"].vals,
          dat["2CHTRPZT"].vals,
          dat["2FRADPYT"].vals,
          dat["2CEAHVPT"].vals,
          dat["2CONDMXT"].vals,
          dat["2UVLSPXT"].vals, 
          dat["2CE00ATM"].vals, 
          dat["2CE01ATM"].vals, 
          dat["2FEPRATM"].vals,
          dat["2SPINATM"].vals,
           dat["2IMINATM"].vals,
           dat["2LVPLATM"].vals,
           dat["2SPHVATM"].vals,
           dat["2IMHVATM"].vals,
           dat["2SMTRATM"].vals,
           dat["2FE00ATM"].vals
          ]

column_names = ["Chandra Time (seconds since 1998 Jan 1, 00:00:00 TT, or MJD 50814.0 TT)",
                "Human-readable Time",
               "PMT1 EED Temperature 2PMT1T (C)",
               "PMT2 EED Temperature 2PMT2T (C)",
               "OutDet1 Temperature 2DTSTATT (C)",
               "OutDet1 Temperature 2DCENTRT (C)",
               "FEA Box Temperature 2FHTRMZT (C)",
               "FEA Box Temperature 2CHTRPZT (C) (C)",
               "+Y EED Temperature 2FRADPYT (C)",
               "-Y EED Temperature 2CEAHVPT (C)",
               "Conduit Temperature 2CONDMXT (C)",
                "Snout Temperature 2UVLSPXT (C)", 
               "CEA HK Temperature 1 (C)", 
               "CEA HK Temperature 2 (C)",
               "FEA PreAmp (C)",
                "Spectroscopic Detector 2SPINATM (C)",
                "Imager Detector 2IMINATM (C)",
                "LVPS 2LVPLATM (C)",
                "Spectroscopic HVPS 2SPHVATM (C)",
                "Imager HVPS 2IMHVATM (C)",
                "Selected Motor 2SMTRATM (C)",
                "FEA 2FE00ATM (C)"               ]

In [12]:
a = pd.DataFrame(data=columns).T
a.columns=column_names
a

,"Chandra Time (seconds since 1998 Jan 1, 00:00:00 TT, or MJD 50814.0 TT)",Human-readable Time,PMT1 EED Temperature 2PMT1T (C),PMT2 EED Temperature 2PMT2T (C),OutDet1 Temperature 2DTSTATT (C),OutDet1 Temperature 2DCENTRT (C),FEA Box Temperature 2FHTRMZT (C),FEA Box Temperature 2CHTRPZT (C) (C),+Y EED Temperature 2FRADPYT (C),-Y EED Temperature 2CEAHVPT (C),...,CEA HK Temperature 1 (C),CEA HK Temperature 2 (C),FEA PreAmp (C),Spectroscopic Detector 2SPINATM (C),Imager Detector 2IMINATM (C),LVPS 2LVPLATM (C),Spectroscopic HVPS 2SPHVATM (C),Imager HVPS 2IMHVATM (C),Selected Motor 2SMTRATM (C),FEA 2FE00ATM (C)
0,7.14658e+08,2020-08-24 12:01:56+00:00,24.6122,27.6084,14.1407,19.6715,10.3368,35.6503,7.4541,37.1193,...,None,None,None,None,None,None,None,None,None,None
1,7.14658e+08,2020-08-24 12:07:24+00:00,24.6122,27.6084,13.8434,19.6715,10.3368,35.6503,7.4541,37.1193,...,None,None,None,None,None,None,None,None,None,None
2,7.14658e+08,2020-08-24 12:12:52+00:00,24.6122,27.6084,13.8434,19.8806,10.3368,35.6503,7.19376,37.1193,...,None,None,None,None,None,None,None,None,None,None
3,7.14659e+08,2020-08-24 12:18:20+00:00,24.6122,27.6084,13.8434,20.1679,10.3368,35.6503,7.05356,37.1193,...,None,None,None,None,None,None,None,None,None,None
4,7.14659e+08,2020-08-24 12:23:48+00:00,24.6122,27.6084,13.8434,20.1941,10.3368,35.6503,7.05356,37.1193,...,None,None,None,None,None,None,None,None,None,None
5,7.14659e+08,2020-08-24 12:29:16+00:00,24.6122,27.6084,13.8434,20.1941,10.0209,35.6503,7.05356,37.213,...,None,None,None,None,None,None,None,None,None,None
6,7.1466e+08,2020-08-24 12:34:44+00:00,24.6122,27.7018,13.8434,20.1941,9.91562,35.6503,7.05356,37.8685,...,None,None,None,None,None,None,None,None,None,None
7,7.1466e+08,2020-08-24 12:40:12+00:00,24.6122,27.7018,13.8434,20.1941,9.91562,35.6503,7.05356,37.8685,...,None,None,None,None,None,None,None,None,None,None
8,7.1466e+08,2020-08-24 12:45:40+00:00,24.6706,27.6707,13.8434,20.1941,9.91562,36.1246,7.05356,37.8685,...,None,None,None,None,None,None,None,None,None,None
9,7.14661e+08,2020-08-24 12:51:08+00:00,24.6706,27.8264,13.8434,20.1941,9.91562,36.3799,7.05356,37.8685,...,None,None,None,None,None,None,None,None,None,None


In [13]:
a.to_excel('daveC_first_anomaly.xlsx')

In [21]:
eventdate

[datetime.datetime(2020, 8, 24, 3, 45, 54, 967615, tzinfo=<matplotlib.dates._UTC object at 0x7f86d8520f60>)]

In [22]:
fetch.data_source.set('maude') # just use ska for this
hrc_i_turnon_msids = ['2IMTPAST', '2IMBPAST', '2IMHBLV', '2IMHVLV', '2LLDIALV', '2RSRFALV', '2GRDVALV', '2TLEV1RT']
dat = fetch.get_telem(hrc_i_turnon_msids,start='2020:272', sampling='full')

Fetching full-resolution data for MSIDS=['2IMTPAST', '2IMBPAST', '2IMHBLV', '2IMHVLV', '2LLDIALV', '2RSRFALV', '2GRDVALV', '2TLEV1RT']
  from 2020:272:12:00:00.000 to 2020:324:15:19:11.354


OSError: request failed with status=502 for URL=https://telemetry.cfa.harvard.edu/maude/mrest/sysinfo.json and text=<html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.11.13</center>
</body>
</html>


In [ ]:
columns = [dat["2IMTPAST"].times, 
           mdate.num2date(hrc.convert_chandra_time(dat["2IMTPAST"].times)),        
          dat["2IMTPAST"].vals, 
          dat["2IMBPAST"].vals,
          dat["2IMHBLV"].vals,
          dat["2LLDIALV"].vals,
          dat["2RSRFALV"].vals,
          dat["2GRDVALV"].vals,
          dat["2TLEV1RT"].vals]

column_names = ["Chandra Time (seconds since 1998 Jan 1, 00:00:00 TT, or MJD 50814.0 TT)",
                "Human-readable Time",
                "2IMTPAST",
               "2IMBPAST",
               "2IMHBLV",
               "2LLDIALV",
               "2RSRFALV",
               "2GRDVALV",
               "2TLEV1RT"]

In [ ]:
a = pd.DataFrame(data=columns).T
a.columns=column_names
a

In [ ]:
a.to_excel('hrcI_turnon.xlsx')

In [ ]:
fetch.data_source.set('maude') 
hrc_i_turnon_msids = ['2IMTPAST', '2IMBPAST', '2IMHBLV', '2IMHVLV', '2LLDIALV', '2RSRFALV', '2GRDVALV', '2TLEV1RT']
dat = fetch.get_telem(hrc_i_turnon_msids,start='2020:272', sampling='full')

In [ ]:
fetch.data_source.set('maude') 
hrc_s_turnon_msids = ['2SPTPAST', '2SPBPAST', '2SPHBLV', '2SPHVLV', '2LLDIALV', '2RSRFALV', '2GRDVALV', '2TLEV1RT']
dat = fetch.get_telem(hrc_s_turnon_msids,start='2020:299', sampling='full')

In [14]:
columns = [dat["2SPTPAST"].times, 
           mdate.num2date(hrc.convert_chandra_time(dat["2SPTPAST"].times)),        
          dat["2SPTPAST"].vals, 
          dat["2SPBPAST"].vals,
          dat["2SPHBLV"].vals,
          dat["2LLDIALV"].vals,
          dat["2RSRFALV"].vals,
          dat["2GRDVALV"].vals,
          dat["2TLEV1RT"].vals]

column_names = ["Chandra Time (seconds since 1998 Jan 1, 00:00:00 TT, or MJD 50814.0 TT)",
                "Human-readable Time",
                "2SPTPAST",
               "2SPBPAST",
               "2SPHBLV",
               "2LLDIALV",
               "2RSRFALV",
               "2GRDVALV",
               "2TLEV1RT"]

In [17]:
a = pd.DataFrame(data=columns).T
a.columns=column_names

In [16]:
a.to_excel('hrcS_turnon.xlsx')